# AMICI Python example "steadystate"
Example using [model_steadystate_scaled.sbml] model to demonstrate and test SBML import and Amici Python interface.

In [1]:
sbml_file = 'model_steadystate_scaled.sbml'
model_name = 'model_steadystate_scaled'
model_output_dir= 'model_steadystate_scaled'

### The example model

In [2]:
import libsbml
SBMLreader = libsbml.SBMLReader()
sbml_doc = SBMLreader.readSBML(sbml_file)
sbml_model = sbml_doc.getModel()

print('Species: ', [s.getId() for s in sbml_model.getListOfSpecies()])

print('\nReactions:')
for reaction in sbml_model.getListOfReactions():
    reactants = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfReactants()])
    products  = ' + '.join(['%s %s'%(int(r.getStoichiometry()) if r.getStoichiometry() > 1 else '', r.getSpecies()) for r in reaction.getListOfProducts()])
    reversible = '<' if reaction.getReversible() else ''
    print('%3s: %10s %1s->%10s\t\t[%s]' % (reaction.getId(), 
                        reactants,
                        reversible,
                        products,
                        libsbml.formulaToL3String(reaction.getKineticLaw().getMath())))


Species:  ['x1', 'x2', 'x3']

Reactions:
 r1:       2 x1  ->        x2		[p1 * x1^2]
 r2:   x1 +  x2  ->        x3		[p2 * x1 * x2]
 r3:         x2  ->      2 x1		[p3 * x2]
 r4:         x3  ->  x1 +  x2		[p4 * x3]
 r5:         x3  ->          		[k4 * x3]
 r6:             ->        x1		[p5]


## Importing an SBML model, compiling and generating an AMICI module

In [3]:
import amici

import os
import sys
import numpy as np
import matplotlib.pyplot as plt

def createModule(sbml_file, model_name, model_output_dir):
    """Create Python module from SBML model"""
    sbmlImporter = amici.SbmlImporter(sbml_file)
    sbml = sbmlImporter.sbml
    
    observables = amici.assignmentRules2observables(sbml, filter=lambda variableId: 
                                                    variableId.startswith('observable_') and not variableId.endswith('_sigma'))
    
    print(observables)
    
    sbmlImporter.sbml2amici(model_name, model_output_dir, verbose=True,
                            observables=observables,
                            constantParameters=['k4'],
                            sigmas={'observable_x1withsigma': 'observable_x1withsigma_sigma'})

createModule(sbml_file, model_name, model_output_dir)


{'observable_x1': 'x1', 'observable_x2': 'x2', 'observable_x3': 'x3', 'observable_x1_scaled': 'scaling_x1 * x1', 'observable_x2_offsetted': 'offset_x2 + x2', 'observable_x1withsigma': 'x1'}
running build_ext
building 'model_steadystate_scaled/_model_steadystate_scaled' extension
swigging swig/model_steadystate_scaled.i to swig/model_steadystate_scaled_wrap.cpp
swig -python -c++ -modern -outdir model_steadystate_scaled -I/home/dweindl/.local/lib/python3.6/site-packages/amici/swig -I/home/dweindl/.local/lib/python3.6/site-packages/amici/include -o swig/model_steadystate_scaled_wrap.cpp swig/model_steadystate_scaled.i
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.6-sXpGnM/python3.6-3.6.3=. -specs=/usr/share/dpkg/no-pie-compile.specs -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/dweindl/src/AMICI-devel/python/examples/example_steadystate/model_steadystate_scaled -I/home/dweindl/.loc

To use python-generated model in matlab, run:
```
modelName = '';
modelDir = '';
amimodel.compileAndLinkModel(modelName, modelDir, [], [], [], []);
amimodel.generateMatlabWrapper(3, 6, 8, 1, 0, 0, [], [ modelDir '/simulate_test.m'], modelName, 'lin', 1, 1);
```

## Running simulations and analyzing results

In [4]:
sys.path.insert(0, os.path.abspath(model_output_dir))
import model_steadystate_scaled as modelModule

model = modelModule.getModel()

# show default parameters
print('Parameters:', np.array(model.getParameters()))

# simulation using default parameters
model.setTimepoints(amici.DoubleVector(np.linspace(0, 60, 60))) 
solver = model.getSolver()
rdata = amici.runAmiciSimulation(model, solver)

print()
#np.set_printoptions(threshold=8, edgeitems=2)
for key, value in rdata.items():
    print('%12s: ' % key, value)



ModuleNotFoundError: No module named '_model_steadystate_scaled'

### Plotting tractories

In [ ]:
def plotStateTrajectories(rdata):
    for ix in range(rdata['x'].shape[1]):
        plt.plot(rdata['t'], rdata['x'][:, ix], label='$x_%d$' % ix)
        plt.xlabel('$t$ (s)')
        plt.ylabel('$x_i(t)$ (mmol/ml)')
        plt.legend()
        plt.title('State trajectories')
    plt.show()
    
def plotObservableTrajectories(rdata):
    for iy in range(rdata['y'].shape[1]):
        plt.plot(rdata['t'], rdata['y'][:, iy], label='$y_%d$' % iy)
        plt.xlabel('$t$ (s)')
        plt.ylabel('$y_i(t)$ (AU)')
        plt.legend()
        plt.title('Observables')
    
    plt.show()
    sys.path.insert(0, 'test')

plotStateTrajectories(rdata)
plotObservableTrajectories(rdata)

### Computing likelihood

In [ ]:
model = modelModule.getModel()
model.setTimepoints(amici.DoubleVector(np.linspace(0, 10, 11))) 
solver = model.getSolver()
rdata = amici.runAmiciSimulation(model, solver)

edata = amici.ExpData(model.get())
edata.my = amici.DoubleVector(rdata['y'].flatten())
edata.sigmay = amici.DoubleVector(np.ones(shape=rdata['y'].shape).flatten())
rdata = amici.runAmiciSimulation(model, solver, edata)

print('Loglikelihood %f' % rdata['llh'])

## Forward sensitivity analysis

In [ ]:
model = modelModule.getModel()
model.setTimepoints(amici.DoubleVector(np.linspace(0, 10, 11))) 
model.requireSensitivitiesForAllParameters()
model.setParameterScale(amici.AMICI_SCALING_NONE)

solver = model.getSolver()
solver.setSensitivityMethod(amici.AMICI_SENSI_FSA)
solver.setSensitivityOrder(amici.AMICI_SENSI_ORDER_FIRST)

rdata = amici.runAmiciSimulation(model, solver)

for key, value in rdata.items():
    if key.startswith('s'):
        print('%12s: ' % key, value)


## Adjoint sensitivity analysis

In [ ]:
model = modelModule.getModel()
p_orig = np.array(model.getParameters())
model.setParameterScale(amici.AMICI_SCALING_NONE)
model.setTimepoints(amici.DoubleVector(np.linspace(0, 10, 11))) 
solver = model.getSolver()
solver.setMaxSteps(10**4)

# simulate time-course for artificial data
rdata = amici.runAmiciSimulation(model, solver)
edata = amici.ExpData(model.get())
edata.fixedParameters = model.getFixedParameters()
edata.my = amici.DoubleVector(rdata['y'].flatten())
# set sigma to 1.0 except for observable 5, so that p[7] is used instead
sigmay = np.ones(shape=rdata['y'].shape)
sigmay[:, 5] = np.nan
edata.sigmay = amici.DoubleVector(sigmay.flatten())

# enable sensitivities
solver.setSensitivityMethod(amici.AMICI_SENSI_ASA)
solver.setSensitivityOrder(amici.AMICI_SENSI_ORDER_FIRST)
model.requireSensitivitiesForAllParameters()

# compute adjoint sensitivities
rdata = amici.runAmiciSimulation(model, solver, edata)
print('Noise-free: llh: %f, sllh: %s' % (rdata['llh'], rdata['sllh']))

# Add some noise
edata.my = amici.DoubleVector(np.multiply(rdata['y'], np.random.normal(0.0, 0.01, rdata['y'].shape)).flatten())
#edata.my = amici.DoubleVector((rdata['y'] * 1.1).flatten())
rdata = amici.runAmiciSimulation(model, solver, edata)
print('Some noise: llh: %f, sllh: %s' % (rdata['llh'], rdata['sllh']))

# Check llh sensitivity w.r.t. sigma parameter: 
p = p_orig.copy()
p[7] = 2
model.setParameters(amici.DoubleVector(p.flatten()))
rdata = amici.runAmiciSimulation(model, solver, edata)
print('Some noise: llh: %f, sllh: %s' % (rdata['llh'], rdata['sllh']))

## Finite differences gradient check

In [ ]:
from scipy.optimize import check_grad
def func(x0, symbol='llh', x0full=None, plist=[], verbose=False):
    p = x0[:]
    if len(plist):
        p = x0full[:]
        p[plist] = x0
    verbose and print('f: p=%s' % p)
    
    old_parameters = model.getParameters()
    solver.setSensitivityOrder(amici.AMICI_SENSI_ORDER_NONE)
    model.setParameters(amici.DoubleVector(p))
    rdata = amici.runAmiciSimulation(model, solver, edata)
    
    model.setParameters(old_parameters)
    
    res = np.sum(rdata[symbol])
    verbose and print(res)
    return res

def grad(x0, symbol='llh', x0full=None, plist=[], verbose=False):
    p = x0[:]
    if len(plist):
        model.setParameterList(amici.IntVector(plist))
        p = x0full[:]
        p[plist] = x0
    else:
        model.requireSensitivitiesForAllParameters()
    verbose and print('g: p=%s' % p)
    
    old_parameters = model.getParameters()
    solver.setSensitivityMethod(amici.AMICI_SENSI_FSA)
    solver.setSensitivityOrder(amici.AMICI_SENSI_ORDER_FIRST)
    model.setParameters(amici.DoubleVector(p))
    rdata = amici.runAmiciSimulation(model, solver, edata)
    
    model.setParameters(old_parameters)

    res = rdata['s%s' % symbol]
    if not isinstance(res, float):
        if len(res.shape) == 3:
            res = np.sum(res, axis=(0, 2))
    verbose and print(res)
    return res

err_norm = check_grad(func, grad, p, 'llh')
print('sllh: |error|_2: %f' % err_norm)
# assert err_norm < 1e-6
print()

for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], 'llh', p, [ip])
    print('sllh: p[%d]: |error|_2: %f' % (ip, err_norm))

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], 'y', p, [ip])
    print('sy: p[%d]: |error|_2: %f' % (ip, err_norm))

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], 'x', p, [ip])
    print('sx: p[%d]: |error|_2: %f' % (ip, err_norm))

print()
for ip in range(model.np()):
    plist = [ip]
    p = p_orig.copy()
    err_norm = check_grad(func, grad, p[plist], 'sigmay', p, [ip])
    print('ssigmay: p[%d]: |error|_2: %f' % (ip, err_norm))
